In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import os
# import shutil
# from tqdm import tqdm

# def shift_files(source_dir, destination_dir, num_files):
#     source_dir = os.path.join(source_dir, '')
#     destination_dir = os.path.join(destination_dir, '')

#     files = os.listdir(source_dir)
#     files.sort(key=lambda file: os.path.getctime(os.path.join(source_dir, file)))

#     shifted_files = []

#     # Using tqdm for progress bar
#     for i in tqdm(range(min(num_files, len(files))), desc="Shifting files"):
#         file_to_move = os.path.join(source_dir, files[i])
#         shutil.move(file_to_move, os.path.join(destination_dir, files[i]))
#         shifted_files.append(files[i])

#     return shifted_files

# def delete_files(source_dir, files_to_delete):
#     source_dir = os.path.join(source_dir, '')

#     # Using tqdm for progress bar
#     for file in tqdm(files_to_delete, desc="Deleting files"):
#         os.remove(os.path.join(source_dir, file))

# source_dir = input("Enter the source directory: ")
# destination_dir = input("Enter the destination directory: ")
# num_files = int(input("Enter the number of files to shift: "))

# shifted_files = shift_files(source_dir, destination_dir, num_files)
# delete_files(source_dir, shifted_files)


In [ ]:
# def delete_files(source_dir, files_to_delete):
#     source_dir = os.path.join(source_dir, '')

#     # Using tqdm for progress bar
#     for file in tqdm(files_to_delete, desc="Deleting files"):
#         os.remove(os.path.join(source_dir, file))

In [ ]:
# delete_files("/content/drive/MyDrive/Final_Video" ,  final_caption_df["filename"])

Deleting files: 100%|██████████| 164/164 [00:00<00:00, 521.09it/s]


In [ ]:
!git clone https://github.com/PKU-YuanGroup/Video-LLaVA

fatal: destination path 'Video-LLaVA' already exists and is not an empty directory.


In [ ]:
cd Video-LLaVA

/content/Video-LLaVA


In [ ]:
!pip install --upgrade pip  # enable PEP 660 support
!pip install -e .
!pip install -e ".[train]"
!pip install flash-attn --no-build-isolation
!pip install decord opencv-python git+https://github.com/facebookresearch/pytorchvideo.git@28fe037d212663c6a24f373b94cc5d478c8c1a1d

In [ ]:
#%%capture
#git clone https://github.com/PKU-YuanGroup/Video-LLaVA
#cd Video-LLaVA
!conda create -n videollava python=3.10 -y
!conda activate videollava

/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found


In [ ]:
%%capture
!pip install --upgrade pip setuptools
!pip install flit
!apt install python3.10-venv
!python3.10 -m venv videollava
!source videollava/bin/activate
!pip install peft
!pip install -e .
!pip install --upgrade pip
!pip install -e ".[train]"
!pip install flash-attn --no-build-isolation
!pip install decord opencv-python git+https://github.com/facebookresearch/pytorchvideo.git@28fe037d212663c6a24f373b94cc5d478c8c1a1d


In [ ]:
import torch

In [ ]:
!pip install -r /content/Video-LLaVA/llava.egg-info/requires.txt

ERROR: Invalid requirement: '[train]' (from line 24 of /content/Video-LLaVA/llava.egg-info/requires.txt)


In [ ]:
import transformers
print(transformers.__version__)

4.31.0


In [ ]:

from llava.constants import X_TOKEN_INDEX, DEFAULT_X_TOKEN

[2023-12-15 05:47:13,754] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


#ERROR AAYEGA NOW RESTART RUNTIME FIR KARO

In [ ]:



from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_X_token, get_model_name_from_path, KeywordsStoppingCriteria

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:

# device_map = {
#     "transformer.word_embeddings": "cpu",
#     "transformer.word_embeddings_layernorm": "cpu",
#     "lm_head": "cpu",
#     "transformer.h": "cpu",
#     "transformer.ln_f": "cpu",
# }


# quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)


In [ ]:
# import pandas as pd
# final_df = pd.DataFrame(columns=['filename', 'caption'])

In [ ]:
# import pandas as pd
# mera_df = pd.read_csv('/content/drive/MyDrive/Final_Datasets_6dec/Train_Sample_withBLIP_9910.csv')

In [ ]:
import os
import torch
import logging
from tqdm import tqdm

class VideoCaptionGenerator:
    _instance = None

    @classmethod
    def initialize(cls, model_path, device='cuda', load_4bit=True, load_8bit=False):
        if cls._instance is None:
            cls._instance = cls(model_path, device, load_4bit, load_8bit)
        return cls._instance

    def __init__(self, model_path, device, load_4bit, load_8bit):
        logging.getLogger("transformers").setLevel(logging.ERROR)
        self.model_name = get_model_name_from_path(model_path)
        self.tokenizer, self.model, self.processor, self.context_len = load_pretrained_model(model_path, None, self.model_name, load_8bit, load_4bit, device=device)
        self.device = device
        self.video_processor = self.processor['video']
        self.conv_mode = "llava_v1"
        self.roles = conv_templates[self.conv_mode].copy().roles

    def generate_caption(self, video, company_value):

        video_tensor = self.video_processor(video, return_tensors='pt')['pixel_values']
        if type(video_tensor) is list:
            tensor = [video.to(self.model.device, dtype=torch.float16) for video in video_tensor]
        else:
            tensor = video_tensor.to(self.model.device, dtype=torch.float16)
        key = ['video']

        inp = f'''This video was posted by the company {company_value} on twitter. Generate caption accordingly'''
        print(f"{self.roles[1]}: {inp}")
        inp = DEFAULT_X_TOKEN['VIDEO'] + '\n' + inp
        conv = conv_templates[self.conv_mode].copy()
        conv.append_message(conv.roles[0], inp)
        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()
        input_ids = tokenizer_X_token(prompt, self.tokenizer, X_TOKEN_INDEX['VIDEO'], return_tensors='pt').unsqueeze(0).to(self.device)
        stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
        keywords = [stop_str]
        stopping_criteria = KeywordsStoppingCriteria(keywords, self.tokenizer, input_ids)

        with torch.inference_mode():
            output_ids = self.model.generate(
                input_ids,
                images=[tensor, key],
                do_sample=True,
                temperature=0.1,
                max_new_tokens=1024,
                use_cache=True,
                stopping_criteria=[stopping_criteria])

        outputs = self.tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
        return outputs


In [ ]:
results = []

In [ ]:
import pandas as pd
mera_df = pd.read_csv("/content/drive/MyDrive/Task2/Test_Company/video_url_with_id.csv")

In [ ]:
mera_df.shape

(3943, 22)

In [ ]:
done_df = pd.read_csv("/content/drive/MyDrive/Task2/Test_Company/Task2_Company_llava2_video_ccaption.csv")

In [ ]:
done_df

,filename,caption
0,6.mp4,The video starts with a man in a blue shirt si...
1,7.mp4,Error generating caption
2,10.mp4,The video starts with a man playing the drums ...
3,14.mp4,The video showcases the beauty of the natural ...
4,16.mp4,A man in a suit is seen speaking to the camera...
...,...,...
3938,9986.mp4,The video shows a man wearing a suit and tie s...
3939,9989.mp4,"The video features a man in a wrestling ring, ..."
3940,9991.mp4,The video shows a man in a suit interviewing a...
3941,9992.mp4,The video features a man in a black shirt who ...


In [ ]:
ids=[]
for i in done_df["filename"]:
     ids.append(i.replace(".mp4", ""))

In [ ]:
ids_int = list(map(int, ids))
len(ids)

3943

In [ ]:
unique_ids = set(ids_int)
count_unique_ids = len(unique_ids)

print(count_unique_ids)

3943


In [ ]:
mera_df = mera_df[~mera_df['id'].isin(ids_int)]

In [ ]:
len(mera_df)

0

In [ ]:
def main():
    directory_path = '/content/drive/MyDrive/Task2/Test_Company/Company_Test_Videos'
    model_path = 'LanguageBind/Video-LLaVA-7B'
    device = 'cuda'

    # Initialize and load the model only once
    caption_generator = VideoCaptionGenerator.initialize(model_path, device)

    # Iterate over each video in the directory


    for filename in tqdm(os.listdir(directory_path), desc="Processing videos", unit="video"):
        if filename.endswith(".mp4"):
            video_path = os.path.join(directory_path, filename)
            company = filename.replace(".mp4", "")
            #print(company)
            company_value = mera_df[mera_df['id'] == int(company)]["username_x"].values[0]



           # inp = f'''This video was posted by the company {company} on twitter. Generate caption accordingly'''
            #print(f"Generating caption for {filename}...")
            try:
                caption = caption_generator.generate_caption(video_path, company_value)
                print(caption)
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                caption = "Error generating caption"
            #caption = caption_generator.generate_caption(video_path,  company_value)
            #final_df.append({'filename': filename, 'caption': caption}, ignore_index=True)
            results.append({'filename': filename, 'caption': caption})
            #final_df = final_df.append({'filename': filename, 'caption': caption}, ignore_index=True)
            #caption_list
            #print(f"Caption for {filename}: {caption}\n")
            if len(results) % 100 == 0:
                save_to_excel(results)

    # Save the remaining results
    if results:
        save_to_excel(results)

if __name__ == '__main__':
    main()

Processing videos:   0%|          | 0/3943 [00:00<?, ?video/s]

ASSISTANT: This video was posted by the company BBCCasualty on twitter. Generate caption accordingly
['video']


Processing videos:   0%|          | 1/3943 [00:15<17:08:16, 15.65s/video]

The video starts with a man in a blue shirt sitting in front of a camera, talking about a topic. The camera then cuts to a man in a hospital room, who is also talking about the same topic. The man in the hospital room is wearing a blue shirt and is seen sitting in a chair. The video ends with the man in the blue shirt sitting in front of the camera again, still talking about the same topic.</s>


Processing videos:   0%|          | 2/3943 [00:16<7:25:58,  6.79s/video] 

...

The video features a blue background with the word "double" written in white. The video also includes a logo for UCLASoftball.</s>
ASSISTANT: This video was posted by the company ESPNFC on twitter. Generate caption accordingly
['video']


Processing videos:  74%|███████▍  | 2933/3943 [6:35:18<1:24:22,  5.01s/video]

The video features a man wearing a black jacket and a backpack walking on a field, and he is seen talking to the camera. The video also shows a man in a black jacket and a backpack walking on a field, and he is seen talking to the camera. The video also shows a man in a black jacket and a backpack walking on a field, and he is seen talking to the camera.</s>
ASSISTANT: This video was posted by the company BBCPolitics on twitter. Generate caption accordingly
['video']


Processing videos:  74%|███████▍  | 2934/3943 [6:35:29<1:54:38,  6.82s/video]

The video features a man in a suit speaking to the camera while a group of people sit behind him. The man continues to speak while the camera pans around the room. The video seems to be a political speech or debate, with the man in the suit likely being one of the speakers. The setting appears to be a conference room or auditorium, with the audience seated in rows of chairs. The man in the suit is wearing a suit and tie, and his speech seems to be well-received by the audience, who can be heard clapping and cheering in the background. Overall, the video seems to be a political event or gathering, with the man in the suit being one of the key speakers.</s>
ASSISTANT: This video was posted by the company uclagymnastics on twitter. Generate caption accordingly
['video']


In [ ]:
def save_to_excel(data):
    final_caption_df = pd.DataFrame(data)
    final_caption_df.to_csv('/content/drive/MyDrive/Task2/Test_Company/Task2_Company_llava2_video_ccaption2.csv', index=False)
    final_caption_df = pd.DataFrame(results)

1156

In [ ]:

# extract_and_download_videos(merged_df, '/content/drive/MyDrive/Task2/Test_Company/Company_Test_Videos')


In [ ]:
merged_df.columns

Index(['id', 'date_x', 'likes_x', 'username_x', 'media_x',
       'inferred company_x', 'Unnamed: 0', 'date_y', 'likes_y', 'username_y',
       'media_y', 'inferred company_y', 'hour_of_day', 'day_of_week',
       'is_weekend', 'photo_or_video', 'video_url', 'duration', 'views',
       'image_link'],
      dtype='object')

In [ ]:
Media_type=[]
for index, row in merged_df.iterrows():
    if "video" in row['media_x']:
        Media_type.append("video")
    elif "image" in row['media_x']:
         Media_type.append("Image")
    else:
        Media_type.append("gif")

In [ ]:
merged_df["Media_Type"]=Media_type

In [ ]:
def extract_video_info(media):
    video_urls = re.findall(r'url=\'(.*?)\'', media)
    bitrates = list(map(int, re.findall(r'bitrate=(\d+)', media)))

    # Filter out video URLs with mp4 format
    mp4_urls = [url for url in video_urls if 'mp4' in url]
    if len(mp4_urls) > 1:
        selected_url = mp4_urls[bitrates.index(max(bitrates))]
    elif mp4_urls:
        selected_url = mp4_urls[0]
    else:
        selected_url = None
    return selected_url

In [ ]:
video_urls =[]
for i in range(len(merged_df)):
    if "video" in merged_df["media_x"][i]:
        video_urls.append(extract_video_info(merged_df["media_x"][i]))

In [ ]:
video_df = merged_df[merged_df['Media_Type'] == 'video']
video_df["video_urls"] = video_urls

<ipython-input-33-e5b2623a8cdf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_df["video_urls"] = video_urls


In [ ]:
video_df.to_csv("/content/drive/MyDrive/Task2/Test_Company/video_url_with_id.csv", index = False)

In [ ]:
import re
url_pattern = re.compile(r'https?://\S+')
urls = []
for text in merged_df['media_x']:
    matches = url_pattern.findall(text)
    urls.extend(matches)